# Laboratorio de Amazon Transcribe y Amazon Comprehend

En este laboratorio veremos como utilizar Amazon Transcribe para obtener una transcripción de un archivo de audio como puede ser un podcast, una conversación en un contact center o un video, del cual queremos extraer la conversación. Después de hacer esta extracción utilizaremos Amazon Comprehend para analizar la extracción de texto y obtener más información sobre la transcripción.

Para comenzar es necesario escribir el nombre del bucket de S3 que estaremos utilizando en el laboratorio así como el nombre del rol, estos los podemos localizar en la sección de Salidas (Outputs) de [CloudFormation]()

In [ ]:
import boto3
import gzip
import io
import requests
import time
import tarfile
import uuid

bucket_name = 'CHANGE_ME'
comprehend_role_arn = 'CHANGE_ME'
comprehend = boto3.client('comprehend')
transcribe = boto3.client('transcribe')
s3 = boto3.client('s3')

## Amazon Transcribe

Comenzaremos extrayendo la conversación de un archivo de texto, en este caso utilizaremos un podcast de AWS en español.

*Tip: Para experimentar puedes sustituir la URL del código apuntando a otro archivo de audio.*

In [ ]:
media_format = 'mp3'
content_type = 'audio/mpeg3'
t_job_name = str(uuid.uuid4())
key = 'audio/' + t_job_name + '.' + media_format
audio_url = 'https://d3h2ozso0dirfl.cloudfront.net/episodes/EP48-Technical-Containers-1.mp3'
audio_bytes = requests.get(audio_url, allow_redirects=True).content
r_put_object = s3.put_object(Body=audio_bytes, Bucket=bucket_name, ContentLength=len(audio_bytes),
                             ContentType=content_type, Key=key)
media_s3_uri = 's3://' + bucket_name + '/' + key

Ahora iniciaremos la ejecución del proceso de transcripción.

In [ ]:
t_output_key = 'transcripts/' + t_job_name
t_response = transcribe.start_transcription_job(TranscriptionJobName=t_job_name,
                                                Media={'MediaFileUri': media_s3_uri},
                                                MediaFormat=media_format,
                                                OutputBucketName=bucket_name,
                                                OutputKey=t_output_key,
                                                IdentifyLanguage=True)

La transcripción es un proceso asíncrono, por lo que es necesario realizar consultas periódicas o diseñar el flujo para continuar con eventos.

In [ ]:
while True:
    r_status = transcribe.get_transcription_job(TranscriptionJobName=t_job_name)
    if 'TranscriptionJob' in r_status and 'TranscriptionJobStatus' in r_status['TranscriptionJob']:
        status = r_status['TranscriptionJob']['TranscriptionJobStatus']
        if status == 'COMPLETED' or status == 'FAILED':
            print(status)
            break
        else:
            print(status)
            time.sleep(30)

Ahora que la transcripción está finalizada podemos recuperarla con las llaves que proporcionamos al servicio al iniciar el proceso. Unimos las diferentes transcripciones y subimos un archivo de texto al servicio de S3. En esta sección podemos ver los limites del servicio [Comprehend Quotas](https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits.html)

In [ ]:
import json

r_get_object = s3.get_object(Bucket=bucket_name, Key=t_output_key)
transcription = json.load(r_get_object['Body'])
text = ''
if 'results' in transcription:
    for transcript in transcription['results']['transcripts']:
        text += transcript['transcript']

text_bytes = bytearray(text, 'utf-8')
text_key = 'text/' + t_job_name + '.txt'
r_put_object = s3.put_object(Body=text_bytes, Bucket=bucket_name, ContentLength=len(text_bytes),
                             ContentType='text/plain', Key=text_key)

Ahora analizaremos el texto con el servicio de Amazon Comprehend. Para evitar llegar al límite de 5000 bytes utilizaremos las funciones asíncronas.
Primero utilizaremos la detección de entidades.

In [ ]:
entities_key = 'comprehend/entities/' + t_job_name
key_phrases_key = 'comprehend/phrases/' + t_job_name
sentiment_key = 'comprehend/sentiment/' + t_job_name
topics_key = 'comprehend/topics/' + t_job_name

comprehend_role_arn = 'arn:aws:iam::253323635394:role/ComprehendIDRole'
entities_r = comprehend.start_entities_detection_job(
    InputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + text_key,
        'InputFormat': 'ONE_DOC_PER_FILE'
    },
    OutputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + entities_key
    },
    JobName=t_job_name + 'Entities',
    LanguageCode='es',
    DataAccessRoleArn=comprehend_role_arn
)

Ahora buscaremos las frases clave dentro del document.

In [ ]:
key_phrases_r = comprehend.start_key_phrases_detection_job(
    InputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + text_key,
        'InputFormat': 'ONE_DOC_PER_FILE'
    },
    OutputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + key_phrases_key
    },
    JobName=t_job_name + 'KeyPhrases',
    LanguageCode='es',
    DataAccessRoleArn=comprehend_role_arn
)

Ahora ejecutaremos la detección de sentimiento (esta función requiere un tamaño máximo de 5120 bytes, por lo que es muy probable que falle con textos grandes).

In [ ]:
sentiment_r = comprehend.start_sentiment_detection_job(
    InputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + text_key,
        'InputFormat': 'ONE_DOC_PER_FILE'
    },
    OutputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + sentiment_key
    },
    JobName=t_job_name + 'Sentiment',
    LanguageCode='es',
    DataAccessRoleArn=comprehend_role_arn
)

Ahora ejecutaremos la detección de tópicos.

In [ ]:
topics_r = comprehend.start_topics_detection_job(
    InputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + text_key,
        'InputFormat': 'ONE_DOC_PER_FILE'
    },
    OutputDataConfig={
        'S3Uri': 's3://' + bucket_name + '/' + topics_key
    },
    JobName=t_job_name + 'Topics',
    DataAccessRoleArn=comprehend_role_arn
)

Ahora esperemos a que Comprehend termine las tareas asignadas. Si encontramos algún fallo (FAILED), podemos revisarlo a detalle en la consola de [Amazon Comprehend](https://console.aws.amazon.com/comprehend).

In [ ]:
entities_job = None
key_phrases_job = None
sentiment_job = None
topics_job = None

while True:
    response = comprehend.describe_entities_detection_job(JobId=entities_r['JobId'])
    if 'EntitiesDetectionJobProperties' in response and 'JobStatus' in response['EntitiesDetectionJobProperties']:
        status = response['EntitiesDetectionJobProperties']['JobStatus']
        print(status)
        if status == 'COMPLETED' or status == 'FAILED':
            entities_job = response['EntitiesDetectionJobProperties']
            break
        else:
            time.sleep(30)

while True:
    response = comprehend.describe_key_phrases_detection_job(JobId=key_phrases_r['JobId'])
    if 'KeyPhrasesDetectionJobProperties' in response and 'JobStatus' in response['KeyPhrasesDetectionJobProperties']:
        status = response['KeyPhrasesDetectionJobProperties']['JobStatus']
        print(status)
        if status == 'COMPLETED' or status == 'FAILED':
            key_phrases_job = response['KeyPhrasesDetectionJobProperties']
            break
        else:
            time.sleep(30)

while True:
    response = comprehend.describe_sentiment_detection_job(JobId=sentiment_r['JobId'])
    if 'SentimentDetectionJobProperties' in response and 'JobStatus' in response['SentimentDetectionJobProperties']:
        status = response['SentimentDetectionJobProperties']['JobStatus']
        print(status)
        if status == 'COMPLETED' or status == 'FAILED':
            sentiment_job = response['SentimentDetectionJobProperties']
            break
        else:
            time.sleep(30)

while True:
    response = comprehend.describe_topics_detection_job(JobId=topics_r['JobId'])
    if 'TopicsDetectionJobProperties' in response and 'JobStatus' in response['TopicsDetectionJobProperties']:
        status = response['TopicsDetectionJobProperties']['JobStatus']
        print(status)
        if status == 'COMPLETED' or status == 'FAILED':
            topics_job = response['TopicsDetectionJobProperties']
            break
        else:
            time.sleep(30)

Comencemos analizando la respuesta de detección de entidades

In [ ]:
if entities_job and entities_job['JobStatus'] == 'COMPLETED':
    output_key = entities_job['OutputDataConfig']['S3Uri'][6 + len(bucket_name):]
    response = s3.get_object(Bucket=bucket_name, Key=output_key)
    tar_file = gzip.GzipFile(fileobj=io.BytesIO(response['Body'].read()))
    tar = tarfile.open(fileobj=tar_file, mode='r:')
    content = tar.extractfile(tar.getmembers()[0])
    entities = json.loads(content.read())
    max_entities = 10 # Cambiar para mostrar más
    for entity in entities['Entities']:
        if entity['Score'] >= 0.8 and max_entities >= 0:
            print('Texto: {} Tipo: {}'.format(entity['Text'], entity['Type']))
            max_entities -= 1

Ahora analicemos la respuesta de detección de frases clave

In [ ]:
if key_phrases_job and key_phrases_job['JobStatus'] == 'COMPLETED':
    output_key = key_phrases_job['OutputDataConfig']['S3Uri'][6 + len(bucket_name):]
    response = s3.get_object(Bucket=bucket_name, Key=output_key)
    tar_file = gzip.GzipFile(fileobj=io.BytesIO(response['Body'].read()))
    tar = tarfile.open(fileobj=tar_file, mode='r:')
    content = tar.extractfile(tar.getmembers()[0])
    key_phrases = json.loads(content.read())['KeyPhrases']
    max_phrases = 10 # Cambiar para mostrar más
    print(key_phrases)
    for phrase in key_phrases:
        if phrase['Score'] >= 0.8 and max_phrases >= 0:
            print('Texto: {}'.format(phrase['Text']))
            max_phrases -= 1

Ahora verifiquemos la respuesta de detección de sentimiento

In [ ]:
if sentiment_job and sentiment_job['JobStatus'] == 'COMPLETED':
    output_key = sentiment_job['OutputDataConfig']['S3Uri'][6 + len(bucket_name):]
    response = s3.get_object(Bucket=bucket_name, Key=output_key)
    tar_file = gzip.GzipFile(fileobj=io.BytesIO(response['Body'].read()))
    tar = tarfile.open(fileobj=tar_file, mode='r:')
    content = tar.extractfile(tar.getmembers()[0])
    key_phrases = json.loads(content.read())['KeyPhrases']
    max_phrases = 10 # Cambiar para mostrar más
    print(key_phrases)
    for phrase in key_phrases:
        if phrase['Score'] >= 0.8 and max_phrases >= 0:
            print('Texto: {}'.format(phrase['Text']))
            max_phrases -= 1

Finalmente analicemos la respuesta de detección de tópicos, esta respuesta son archivos CSV.

In [ ]:
if topics_job and topics_job['JobStatus'] == 'COMPLETED':
    output_key = topics_job['OutputDataConfig']['S3Uri'][6 + len(bucket_name):]
    response = s3.get_object(Bucket=bucket_name, Key=output_key)
    tar_file = gzip.GzipFile(fileobj=io.BytesIO(response['Body'].read()))
    tar = tarfile.open(fileobj=tar_file, mode='r:')
    for member in tar.getmembers():
        print(member.name)
        print(str(tar.extractfile(member).read(), 'utf-8'))
